# Notebook with examples for Transformations

This Notebook shows how a system can be reduced to a minmal system or transfomed into a cannonical form.

In [ ]:
import numpy as np
from tvsclib.mixed_system import MixedSystem
from tvsclib.strict_system import StrictSystem

from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.transformations.reduction import Reduction
from tvsclib.transformations.input_normal import InputNormal
from tvsclib.transformations.output_normal import OutputNormal

from tvsclib.stage import Stage

import tvsclib.utils as utils
import matplotlib.pyplot as plt

## Create a system

In [ ]:
dims_in =  [2, 1, 2, 1, 5, 2,10, 3, 2, 1, 3, 2, 4, 2, 5,20,30,10,10,10,15]
dims_out = [1, 2, 1, 2, 5, 2, 7, 3, 2, 1, 5, 7, 2, 1, 2,20,30,10,10,10,15]
matrix = np.random.rand(sum(dims_out), sum(dims_in))
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-10)
system = MixedSystem(S)
system_causal = system.causal_system
system_anticausal = system.anticausal_system

The system is already mininmal and balaned, as the identification already did take care of this.

In [ ]:
print("Causal minimal:",system_causal.is_minimal())
print("Anticausal minimal:",system_anticausal.is_minimal())
print("Causal balanced:",system_causal.is_balanced(tolerance=1e-11))
print("Anticausal balanced:",system_anticausal.is_balanced(tolerance=1e-11))

## Input normal

We can now transform it to input normal form

In [ ]:
sys_causal_inp = InputNormal().apply(system_causal)
sys_anticausal_inp = InputNormal().apply(system_anticausal)
print("Causal input normal:",sys_causal_inp.is_input_normal())
print("Anticausal input normal:",sys_anticausal_inp.is_input_normal())

Now check if the systems systems are really equivalent

In [ ]:
print(np.max(np.abs(sys_causal_inp.to_matrix()-system_causal.to_matrix())))
print(np.max(np.abs(sys_anticausal_inp.to_matrix()-system_anticausal.to_matrix())))

Input normality means that the collumns in the reachability matricies $\mathcal{R}_k$ are orthonormal.
This mans that the reachability gramian $\mathcal{R}_k \mathcal{R}_k^\top$ is the identity matrix.

Lets illustrate this here:

In [ ]:
n = 2
gram = sys_anticausal_inp.reachability_matrix(n)@sys_anticausal_inp.reachability_matrix(n).T
print(gram)
np.allclose(gram,np.eye(gram.shape[0]))

When aplying the transformation the stages get copied

In [ ]:
sys_anticausal_inp.stages is system_anticausal.stages

## Output normal

In [ ]:
sys_causal_out = OutputNormal().apply(system_causal)
sys_anticausal_out = OutputNormal().apply(system_anticausal)
print("Causal output normal:",sys_causal_out.is_output_normal())
print("Anticausal output normal:",sys_anticausal_out.is_output_normal())

Output normality means that the collumns in the observability matricies are orthonormal.
This means that the observability gramian $\mathcal{O}_k^\top \mathcal{O}_k$ is the identity matrix.

Lets illustrate this here:

In [ ]:
n = 2
gram = sys_anticausal_out.observability_matrix(n).T@sys_anticausal_out.observability_matrix(n)
print(gram)
np.allclose(gram,np.eye(gram.shape[0]))

## Reduction

First create a nonminimal system.
For this we set try to represent a low rank matrix and set the epsilon to 0. This causes many spurious states 

In [ ]:
dims_in =  [2, 1, 2, 1, 5, 2,10, 3, 2, 1, 3, 2, 4, 2, 5,20,30,10,10,10,15]
dims_out = [1, 2, 1, 2, 5, 2, 7, 3, 2, 1, 5, 7, 2, 1, 2,20,30,10,10,10,15]
rank = 2
matrix_a = np.random.rand(sum(dims_out),rank)@np.random.rand(rank,sum(dims_in))
T = ToeplitzOperator(matrix_a, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=0)
system_a = MixedSystem(S)
print(system_a)

print("rank(matrix_a)=",np.linalg.matrix_rank(matrix_a))

In [ ]:
np.max(abs(matrix_a-system_a.to_matrix()))

Now we apply the reduction and get a minimal system without converting it from a matrix

In [ ]:
system_a_red = Reduction().apply(system_a)
print(system_a_red)
np.max(abs(matrix_a-system_a_red.to_matrix()))

We can also set an $\epsilon$, if we want to remove bigger singular values of the Hankel matrices.

In [ ]:
np.max(abs(matrix_a-system_a_red.to_matrix()))

This reduction might introduced an error.

### Implementation of Output and Input normal

The transformations to input and output normal form use the QR-decomposition.
Here a short illustration of the output normal algorithm is given. the input normal algorithm is similar and it shoud be no problem to understand it if the output normal implementation is clear.

A system is output normal if $$A_k^\top A_k+C_k^\top C_k = 1 $$ for all $k$

This is equivalent to 
$$
\begin{bmatrix}
   A_k^\top & C_k^\top \\
\end{bmatrix}
\begin{bmatrix}
   A_k\\
   C_k
\end{bmatrix}
=1
$$

To obtian this we want to make the matrix $\begin{bmatrix}
   A_k\\
   C_k
\end{bmatrix}$ orthogonal.

This can be done with state transforms of the structure

$$\hat{A}_k=S_{k+1} A_k S_k^{-1}$$
This can be done using the QR factorization. 
For this we look over the stages in descending order.
We already know the state transform $S_{k+1}$ form the previous step. (The inital step of the algorithm does not have a this state transform as the $A$ matrix has vanishing size)

If we apply the QR-factorization to the matrix $\begin{bmatrix}
   S_{k+1} A_k\\
   C_k
\end{bmatrix}$ we obtain the a $Q$ that can be split up as following

$$
\begin{bmatrix}
   S_{k+1} A_k\\
   C_k
\end{bmatrix} = QR = \begin{bmatrix}
   \hat{A}_k\\
   \hat{C}_k
\end{bmatrix} R
$$

We can now see that the matrices $\hat{A}_k$ and $\hat{A}_k$ fullfill the condition.
The $R$ is the transformation matrix $S_k$
Also watch out that the transformation with $S_k^{-1}$ is already implicitly done by the QR-decomposition.

As a last step we transform the matrices of the next stages with $S_k$

$$S_{k} A_{k-1}$$
$$S_{k} B_{k-1} $$

**Some notes on observability, reachability and minimality**

The algorithm produces a observable realization as the observability matrix $𝓞$ consists of orthogonal collumns.

Now the question is if the transformationalso preserves reachability.

For this we have to consider that the reachability matrix is is transformed according to:

$$ \hat{\mathcal{R}} = S_k \mathcal{R} $$

If $S_k$ has full rank and the original system is reachable the new observability matrix $\hat{\mathcal{R}}$ has also full rank and thus the system is observable.

The matrix $S_k$ has full rank iff the matries $\begin{bmatrix}
   A_k\\
   C_k
\end{bmatrix}$
have full rank. This is the case if the system is reachable.

Therefore the system preserves minimality.
If we would like to make the system minimal, use the `Reduction`, as `OutputNormal` and `InputNormal` do not change the state dimensions.


### Implementation of Reduction
The Reduction algorithm uses an algorithm based on the algorithm described in [[1](#Chandrasekaran_2005)].
Using a small extention one can truncate the states corresponding to small singular values. 

First the system is transformed to a reachable and input normal system. Then it is transformed to an minmal and output normal system.
Then an iterative algorithm is used to make the system minimal. The algorithm starts with the the stage $K=k$ and iterates down to $k=1$
Here we use the fact that the singular values of the Hankel matrix $H_k$ can be decomposed into
 $$
 H_k = \mathcal{O}_k \mathcal{R}_k = 
 \begin{bmatrix}
 1& 0 \\
 0& \mathcal{O}_{k+1}
 \end{bmatrix}
 \begin{bmatrix}
 C_k\\
 A_k
 \end{bmatrix}
 \mathcal{R}_k
 .
 $$
As $\mathcal{R}_k$ is orthogonal form the previous transformation to input normal and $\mathcal{O}_{k+1}$ is made orthogonal the previous iteration, the singular values of $H_k$ can be obtianed by computing the singular values of the matrix 
$$ 
\begin{bmatrix}
 C_k\\
 A_k
 \end{bmatrix}
$$
The results can be used to make the $\mathcal{O}_{k}$ orthogonal, which allows us to continue the computation for the next $k$.

### References
[1]:<a id='Chandrasekaran_2005'></a> S. Chandrasekaran, P. Dewilde, M. Gu, T. Pals, X. Sun, A. J. van der Veen, and D. White. “Some Fast Algorithms for Sequentially Semiseparable Representations”. In: SIAM Journal on Matrix Analysis and Applications 27(2) (Jan. 2005). Publisher: Society for Industrial and Applied Mathematics, pp. 341–364. ISSN: 0895-4798. [DOI: 10.1137/S0895479802405884](https://doi.org/10.1137/S0895479802405884)